<a href="https://colab.research.google.com/github/rosafilgueira/PythonCloneDetection/blob/main/Vector_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [ ]:
 %pip install transformers scikit-learn
 !pip install -U sentence-transformers
 !python --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=14a2cb97b190ad7f688e6ccf5d1c59fc88a259d3bb3aad8351b64592c993b75e
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Python 3.8.10


In [149]:
!mv test* repo4/.

In [ ]:
 %pip install transformers datasets tqdm scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [20]:
!rm -rf ./repo4/.ipynb_checkpoints

In [41]:
import ast
import astunparse
from scipy.optimize import linesearch

import torch
import os
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import inspect
import ast
import astunparse


# function to extract source code lines of each function 
def extract_function_source_code(file_name):
    func=[]
    with open(file_name, 'r') as file:
        source_code = file.read() 
        parsed_code = ast.parse(source_code)
        function_list = [node for node in ast.walk(parsed_code) if isinstance(node, ast.FunctionDef)]
        for f_obj in function_list:
             data=astunparse.unparse(f_obj)
             func.append(data)
        return func

def create_repo_embeddings(repo_path):
  tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
  model = AutoModel.from_pretrained("microsoft/graphcodebert-base")
  
 
 # Initialize the list to store the file embeddings
  # Iterate through all the source files in the repository

  files_embedding=[]
  for file in os.listdir(repo_path):
    f_embedding=[]
    # Extract the source code from each file
    functions = extract_function_source_code(os.path.join(repo_path, file))
    for f_code in functions: 
      #print(f_code)
      # Generate the embedding for each function
      code_tokens=tokenizer.tokenize(f_code)
      #print(code_tokens)
      tokens=[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
      #print(tokens)
      tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
      #print(tokens_ids)
      embedding=model(torch.tensor(tokens_ids)[None,:])[0][0,0]
      # Append the embedding to the list
      f_embedding.append(embedding)
    if len(f_embedding) > 0:
        f = torch.mean(torch.stack(f_embedding), dim=0)
        files_embedding.append(f)
     
  # Aggregate the file embeddings to get the repo code embedding
  repo_embedding = torch.mean(torch.stack(files_embedding), dim=0)
  return repo_embedding

repo1_embedding=create_repo_embeddings("./repo")
repo2_embedding=create_repo_embeddings("./repo2")
repo3_embedding=create_repo_embeddings("./repo3")
repo4_embedding=create_repo_embeddings("./repo4")

#cos = torch.nn.CosineSimilarity(dim=0)

output = nn.CosineSimilarity(dim=0, eps=1e-6)(repo1_embedding, repo2_embedding)
print("Cosine Similarity between repo1 and repo2: %s" % output)


output_2 = nn.CosineSimilarity(dim=0, eps=1e-6)(repo1_embedding, repo3_embedding)
print("Cosine Similarity between repo1 and repo3: %s" % output_2)

output_3 = nn.CosineSimilarity(dim=0, eps=1e-6)(repo2_embedding, repo4_embedding)
print("Cosine Similarity between repo2 and repo4: %s" % output_3)

#output = cos(repo1_embedding, repo2_embedding)
#print("Cosine Similarity between repo1 and repo2: %s" % output)
#output2 = cos(repo1_embedding, repo3_embedding)
#print("Cosine Similarity between repo1 and repo3: %s" % output2)
#output3 = cos(repo4_embedding, repo3_embedding)
#print("Cosine Similarity between repo4 and repo3: %s" %output3)


Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

Cosine Similarity between repo1 and repo2: tensor(1.0000, grad_fn=<SumBackward1>)
Cosine Similarity between repo1 and repo3: tensor(0.9037, grad_fn=<SumBackward1>)
Cosine Similarity between repo2 and repo4: tensor(0.9292, grad_fn=<SumBackward1>)
